In [1]:
# %pip install datasets -q
# %pip install transformers -q
# %pip install torch -q
# %pip install seqeval -q
# %pip install evaluate -q
# %pip install accelerate -q

In [2]:
from datasetutils import decode
from iob2converter import iob2_to_dataset
from transformers import AutoModelForTokenClassification

In [3]:
file_path = '../data/TaggedSeparated/German/synopses_02.iob2'

de_ds = iob2_to_dataset(file_path)

ner_feature_fr = de_ds.features['ner_tags']
label_names = ner_feature_fr.feature.names
print(label_names)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-POK', 'I-POK']


In [4]:
words = de_ds[0]['tokens']
labels = de_ds[0]['ner_tags']
print('\n'.join(decode(words, labels, label_names)))

Ash   Pikachu und der Rest der Gang sehen ihre größte Herausforderung entgegen als zwei hinterlistige Diebinnen den geheimnisvollsten und gefährlichsten aller Kristalle Herztropfen rauben wollen Fällt er in ihre Hände ist die Zerstörung der Wasserstadt Altomare unvermeidbar Es beginnt ein atemberaubendes Rennen gegen die Zeit bei dem die letzte Hoffnung auf Latios und Latias ruht die als Hüter des Kristalls mit magischen Kräften ausgestattet sind 
B-PER B-POK   O   O   O    O   O    O     O    O      O               O        O   O    O             O         O   O                 O   O              O     O         O           O      O      O     O  O  O    O     O   O   O          O   B-LOC       I-LOC    O            O  O       O   O               O      O     O   O    O   O   O   O      O        O   B-POK  O   B-POK  O    O   O   O     O   O         O   O         O       O            O    


In [5]:
from transformers import AutoTokenizer

model_id = 'google-bert/bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        padding="max_length", 
        max_length=128,
        is_split_into_words=True
    )

    all_labels = examples["ner_tags"]

    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels

    return tokenized_inputs

def align_labels_with_tokens(labels, word_ids):
    """
    This function aligns labels with tokens produced by the tokenizer.
    - `-100` is used for special tokens to ignore them during training.
    - If the label is B-XXX, subsequent sub-tokens receive I-XXX.
    """
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            # Convert B-XXX to I-XXX for sub-tokens
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [10]:
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value

tag_to_id = {
	'O': 0,
	'B-PER': 1,
	'I-PER': 2,
	'B-ORG': 3,
	'I-ORG': 4,
	'B-LOC': 5,
	'I-LOC': 6,
	'B-MISC': 7,
	'I-MISC': 8,
	'B-POK': 9,
	'I-POK': 10,
}
id_to_tag = {id: tag for tag, id in tag_to_id.items()}

def iob2_to_datasets(file_path, reference_path):
    """
    Converts an IOB2 file into a DatasetDict with train and validation splits.
    Assumes the input file uses whitespace to separate tokens and tags, and that each sentence is separated by a blank line.
    """
    tokens, ner_tags = [], []
    sentences, sentence_tags = [], []

    label_set = set()
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens and ner_tags:
                    sentences.append(tokens)
                    sentence_tags.append(ner_tags)
                tokens, ner_tags = [], []
            else:
                word, tag = line.split()
                tokens.append(word)
                ner_tags.append(tag)
                label_set.add(tag)

        if tokens and ner_tags:
            sentences.append(tokens)
            sentence_tags.append(ner_tags)

    label_list = list(tag_to_id.keys())
    label_mapping = {label: i for i, label in enumerate(label_list)}

    indexed_tags = [[label_mapping[tag] for tag in tags] for tags in sentence_tags]
    dataset = Dataset.from_dict({"tokens": sentences, "ner_tags": indexed_tags})
    reference_german = iob2_to_dataset(reference_path[0]).remove_columns(["ner_tags_id", "index", "id"])
    reference_french = iob2_to_dataset(reference_path[1]).remove_columns(["ner_tags_id", "index", "id"])
    reference_english = iob2_to_dataset(reference_path[2]).remove_columns(["ner_tags_id", "index", "id"])
    

    features = Features({
        "tokens": Sequence(Value("string")),
        "ner_tags": Sequence(ClassLabel(names=label_list))
    })

    datasets = DatasetDict({
        "train": dataset.cast(features),
        "val_ger": reference_german.cast(features),
        "val_fr": reference_french.cast(features),
        "val_en": reference_english.cast(features),
    })

    return datasets

file_path = '../data/TaggedSeparated/German/synopses_02.iob2'
reference_grp_path = ['../ReferenceText/ReferenceTextGerman.iob2', '../ReferenceText/ReferenceTextFrench.iob2', '../ReferenceText/ReferenceTextEnglish.iob2']

de_ds = iob2_to_datasets(file_path, reference_grp_path)

Casting the dataset:   0%|          | 0/14 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    model_id, num_labels=len(label_names)
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
tokenized_ds = de_ds.map(tokenize_and_align_labels, batched=True)

print(tokenized_ds)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14
    })
    val_ger: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
    val_fr: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
    val_en: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})


In [13]:
import evaluate

metric = evaluate.load("seqeval")

In [14]:
# Rob span-f1 

def toSpans(tags):
    spans = set()
    for beg in range(len(tags)):
        if tags[beg][0] == 'B':
            end = beg
            for end in range(beg+1, len(tags)):
                if tags[end][0] != 'I':
                    break
            spans.add(str(beg) + '-' + str(end) + ':' + tags[beg][2:])
    return spans


def getInstanceScores(predSpans, goldSpans):
    tp = 0
    fp = 0
    fn = 0
    overlap = len(goldSpans.intersection(predSpans))
    tp += overlap
    fp += len(predSpans) - overlap
    fn += len(goldSpans) - overlap
        
    prec = 0.0 if tp+fp == 0 else tp/(tp+fp)
    rec = 0.0 if tp+fn == 0 else tp/(tp+fn)
    f1 = 0.0 if prec+rec == 0.0 else 2 * (prec * rec) / (prec + rec)
    return f1


In [15]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    
    pred_spans, true_spans = toSpans(true_predictions[0]), toSpans(true_labels[0])
    print(pred_spans, true_spans)
    score = getInstanceScores(pred_spans, true_spans)

    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
        "span_f1": score
    }

In [16]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained (
    model_id,
    num_labels=len(label_names),
    id2label={id: label for id, label in enumerate(label_names)},
    label2id={label: id for id, label in enumerate(label_names)},
)
model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


11

In [17]:
from transformers import TrainingArguments
from transformers import Trainer


args = TrainingArguments(
    "mbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [18]:
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=tokenized_ds["train"],
#     eval_dataset=tokenized_ds["validation"],
#     compute_metrics=compute_metrics,
# )

# trainer.train()


In [50]:
import os
import pandas as pd

def fine_tune_models_in_folder(folder_path, output_dir):
    files = os.listdir(folder_path)
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        
        de_ds = iob2_to_datasets(file_path, reference_grp_path)
        
        tokenized_ds = de_ds.map(tokenize_and_align_labels, batched=True)
        
        model = AutoModelForTokenClassification.from_pretrained(
            model_id, num_labels=len(label_names)
        )
        
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=tokenized_ds["train"],
            eval_dataset=tokenized_ds["val_fr"],
            compute_metrics=compute_metrics,

        )

        trainer.predict(tokenized_ds["val_fr"])
        

folder_path = '../data/TaggedSeparated/French'
output_dir = '../Models/m_fr'

fine_tune_models_in_folder(folder_path, output_dir)


Casting the dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'61-64:LOC', '81-87:PER', '121-125:PER', '51-55:MISC', '32-34:LOC', '28-29:ORG', '21-22:ORG', '103-108:MISC', '118-119:MISC', '119-120:ORG', '7-8:ORG', '39-41:MISC', '26-28:ORG', '100-101:LOC', '101-103:MISC', '36-39:MISC', '41-48:ORG', '109-110:ORG', '9-21:MISC', '22-23:ORG', '110-118:POK', '29-31:ORG', '70-73:LOC', '55-61:LOC', '64-70:ORG', '87-91:LOC', '23-24:ORG', '24-25:ORG', '8-9:MISC', '25-26:ORG', '31-32:PER', '48-51:ORG', '120-121:ORG', '91-100:LOC', '73-81:LOC', '34-36:LOC', '6-7:PER'} {'6-8:LOC', '91-96:POK', '9-10:PER', '39-40:PER', '31-32:POK', '30-31:POK', '103-104:PER', '81-82:PER', '122-123:PER'}


  0%|          | 0/1 [00:00<?, ?it/s]

{'7-9:ORG', '19-21:MISC', '9-11:ORG', '114-115:ORG', '18-19:ORG', '96-99:ORG', '11-12:ORG', '87-91:MISC', '94-96:ORG', '84-85:ORG', '30-32:ORG', '52-53:LOC', '71-78:ORG', '32-37:ORG', '86-87:LOC', '39-42:ORG', '4-6:MISC', '99-100:ORG', '113-114:ORG', '93-94:ORG', '100-106:PER', '108-109:ORG', '70-71:LOC', '91-93:ORG', '12-13:ORG', '48-51:MISC', '62-63:LOC', '6-7:ORG', '64-68:MISC', '106-108:ORG', '13-18:ORG', '21-29:ORG', '51-52:LOC', '61-62:LOC', '85-86:ORG', '115-121:POK', '109-111:ORG', '79-84:ORG', '55-56:ORG', '121-122:ORG', '68-69:ORG', '122-123:POK', '2-4:MISC', '56-60:MISC', '63-64:MISC', '42-44:PER', '123-124:MISC', '111-113:ORG', '29-30:PER', '69-70:ORG', '60-61:POK', '53-55:LOC', '37-39:ORG'} {'6-8:LOC', '115-120:POK', '48-49:PER', '102-103:PER', '36-38:POK', '11-12:PER', '38-39:POK'}


KeyError: '0'